In [5]:
#!pip install transformers

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/DIT Data_Science/NLP_Twitter

/content/drive/MyDrive/DIT Data_Science/NLP_Twitter


In [8]:
import csv
import pandas as pd
import numpy as np
import os
import re

from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BasicTokenizer
from transformers import BertModel, BertPreTrainedModel, BertForSequenceClassification
# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

# Torch ML libraries
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings('ignore')


In [55]:
# Set GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
print(device)

cuda


In [11]:
import pandas as pd

# Read the text file into a list of lines
with open('Bambara_v2_dataset.txt') as f:
    lines = f.readlines()

# Split each line into usertext and sentiment
usertexts = []
sentiments = []
for line in lines:
    usertext, sentiment = line.split(';')

    usertexts.append(usertext.strip())
    sentiments.append(sentiment.strip())

# Create a DataFrame with the usertext and sentiment data
data = {'usertext': usertexts, 'sentiment': sentiments}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('Bambara_v2_dataset.csv', index=False)


In [12]:
data = pd.read_csv("Bambara_v2_dataset.csv")

In [13]:
data

,usertext,sentiment
0,iba one deyé mali rap yé ni mi iko ala ete djé...,-1
1,halala a dan,1
2,gnègo den,-1
3,gon den nièguoden,-1
4,halala fosaya,1
...,...,...
3041,valide dah marfa ka flow magni,1
3042,tg fosa ya,1
3043,cool tg c est la loi ibaone fosaya,1
3044,rendez vous le 24 octobre inch allah tg fo saya,1


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3046 entries, 0 to 3045
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   usertext   3046 non-null   object
 1   sentiment  3046 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.7+ KB


In [15]:
data['sentiment'] = data['sentiment'] + 1

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
MODEL_NAME = 'bert-base-multilingual-cased'

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [17]:
import numpy as np
MAX_LEN = 160
# Random seed for reproducibilty
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [18]:
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch
# Misc.
import warnings
warnings.filterwarnings('ignore')
class SentimentDataset(Dataset):
    # Constructor Function
    def __init__(self, texts, sentiments, tokenizer, max_len):
        self.texts = texts
        self.sentiments = sentiments
        self.tokenizer = tokenizer
        self.max_len = max_len

    # Length magic method
    def __len__(self):
        return len(self.texts)

    # get item magic method
    def __getitem__(self, item):
        text = str(self.texts[item])
        target = int(self.sentiments[item])

        # Encoded format to be returned
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'user_text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'sentiment': torch.tensor(target, dtype=torch.long)
        }

In [19]:
df_train, df_test = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

print(df_train.shape, df_val.shape, df_test.shape)

(2436, 2) (305, 2) (305, 2)


In [20]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = SentimentDataset(
        texts=df.usertext.to_numpy(),
        sentiments=df.sentiment.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=0
    )

In [21]:
# Create train, test and val data loaders
BATCH_SIZE = 16
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [22]:
# Examples
data = next(iter(train_data_loader))
print(data.keys())
print(data['user_text'])
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['sentiment'].shape)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


dict_keys(['user_text', 'input_ids', 'attention_mask', 'sentiment'])
['tchiennfoooooo', 'eva i ka nostalgie de bé ne kan', 'malo bobara bah wly il n est connais pas', 'na bana i bé do wairai sôrô ka fô je sais', 'i ni ce', 'hahaha tarzan ya dekoun kadi ayé donc alé bolo djourou kissai dedo', 'bilakoro dagade jougousago de', 'ne koni dabali de bana', 'i ghasigi dogo kana koro ta folo', 'nelson singuinna', 'maria den fama den', 'adan kotey fo saya', 'tg drón walaye bilay one', 'i diabaté kotila', 'adan do fama', 'awo halala tal b an ga tafé']
torch.Size([16, 160])
torch.Size([16, 160])
torch.Size([16])


In [23]:
bert_model = BertModel.from_pretrained(MODEL_NAME)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [24]:
# Build the Sentiment Classifier class
class SentimentClassifier(nn.Module):

    # Constructor class
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    # Forward propagaion class
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask,
          return_dict=False

        )
        #  Add a dropout layer
        output = self.drop(pooled_output)
        return self.out(output)

In [25]:
# Instantiate the model and move to classifier
model = SentimentClassifier(torch.numel(data["sentiment"].unique()))
model = model.to(device)

In [26]:
print(torch.numel(data["sentiment"].unique()))

3


In [27]:
# Number of hidden units
print(bert_model.config.hidden_size)

768


In [28]:
# Number of iterations
EPOCHS = 10

# Optimizer Adam
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Set the loss function
loss_fn = nn.CrossEntropyLoss().to(device)

In [29]:
# Function for a single training iteration
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:

        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["sentiment"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _,preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

        # Backward prop
        loss.backward()

        # Gradient Descent
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

In [30]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()

    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["sentiment"].to(device)

            # Get model ouptuts
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

In [31]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

    # Show details
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print("-" * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(df_train)
    )

    print(f"Train loss {train_loss} accuracy {train_acc}")

    # Get model performance (accuracy and loss)
    val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device,
        len(df_val)
    )

    print(f"Val   loss {val_loss} accuracy {val_acc}")
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    # If we beat prev performance
    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'best_model_state.bin')
        best_accuracy = val_acc


Epoch 1/10
----------
Train loss 1.0146609821350745 accuracy 0.5414614121510674
Val   loss 0.9908052265644074 accuracy 0.5377049180327869

Epoch 2/10
----------
Train loss 1.0010345079540428 accuracy 0.550903119868637
Val   loss 0.9832690477371215 accuracy 0.5377049180327869

Epoch 3/10
----------
Train loss 1.0035285988664315 accuracy 0.5521346469622331
Val   loss 0.991373386979103 accuracy 0.5377049180327869

Epoch 4/10
----------
Train loss 0.9981632824816735 accuracy 0.5533661740558292
Val   loss 0.9902603507041932 accuracy 0.5377049180327869

Epoch 5/10
----------
Train loss 0.9971525622349159 accuracy 0.5537766830870279
Val   loss 0.9880752772092819 accuracy 0.5377049180327869

Epoch 6/10
----------
Train loss 0.9997919813480253 accuracy 0.5533661740558292
Val   loss 0.9838791996240616 accuracy 0.5377049180327869

Epoch 7/10
----------
Train loss 0.9668521374658822 accuracy 0.548440065681445
Val   loss 0.9193140178918838 accuracy 0.5377049180327869

Epoch 8/10
----------
Train lo

In [32]:
def get_predictions(model, data_loader):
    model = model.eval()

    review_texts = []
    predictions = []
    prediction_probs = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:
            texts = d["user_text"]
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["sentiment"].to(device)

            # Get outouts
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            _, preds = torch.max(outputs, dim=1)

            review_texts.extend(texts)
            predictions.extend(preds)
            prediction_probs.extend(outputs)
            real_values.extend(targets)

    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()

    return review_texts, predictions, prediction_probs, real_values


In [33]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
    model,
    test_data_loader
)

In [41]:
report = classification_report(y_test, y_pred, output_dict=True)
dfReport = pd.DataFrame(report).transpose()
myList = dfReport.index.tolist()
myList

['0', '1', '2', 'accuracy', 'macro avg', 'weighted avg']

In [44]:
class_names = df['sentiment'].unique()

In [45]:
print(classification_report(y_test, y_pred, target_names=class_names))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        66
           1       0.42      0.65      0.51        88
           0       0.70      0.78      0.74       151

    accuracy                           0.57       305
   macro avg       0.37      0.48      0.42       305
weighted avg       0.47      0.57      0.51       305



## Prédire avec des valeurs réels

In [36]:
review_text = ""

In [53]:
encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
)

In [56]:
#device = torch.device( "cpu")
input_ids = encoded_review['input_ids'].to(device)
attention_mask = encoded_review['attention_mask'].to(device)

output = model(input_ids, attention_mask)
_, prediction = torch.max(output, dim=1)

print(f'Review text: {review_text}')
print(f'Sentiment  : {class_names[prediction]}')

Review text: 
Sentiment  : 0
